In [1]:
import pandas as pd

In [2]:
import os

In [3]:
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
%matplotlib inline

In [4]:
os.getcwd()

'C:\\Users\\chris\\data_science_projects\\data-question-4-affordable-housing-green-dogs'

In [5]:
bp = pd.read_csv('Building_Permits_Issued.csv')

C:\Users\chris\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
bp

,Permit #,Permit Type Description,Permit Subtype Description,Parcel,Date Entered,Date Issued,Const. Cost,Address,City,State,Zip,Subdivision/Lot,Contact,Permit Type,Permit Subtype,IVR Trk#,Purpose,Council Dist,Census Tract,Mapped Location
0,2018002927,Building Residential - New,Single Family Residence,081160R00100CO,01/18/2018,02/12/2018,206150.0,1017A MONROE ST,NASHVILLE,TN,37208,UNIT A 1017 MONROE,"TRISTAR CONSTRUCTION, INC",CARN,CAA01R301,3439281,to construct 1830SF single family residence wi...,19.0,37019400.0,"1017A MONROE ST\nNASHVILLE, TN 37208\n(36.1740..."
1,201617929,Building Use & Occupancy,"Tents, Stages",09215014800,04/19/2016,04/21/2016,2000.0,2300 WEST END AVE,NASHVILLE,TN,37203,N W CORNER WEST END AVE & 23RD AVE N,Music City Tents & Events Llc,CAUO,CAZ06A001,2116751,PERMIT TO ERECT 1:40X85 TENT. THIS TENT IS BEI...,21.0,37016500.0,"2300 WEST END AVE\nNASHVILLE, TN 37203\n(36.14..."
2,2016049270,Building Sign Permit,Sign - Ground / Wall Signs,11909014000,09/14/2016,09/14/2016,15000.0,2802 NOLENSVILLE PIKE,NASHVILLE,TN,37211,PT LOT 2 PATTERSON SUB,BOZMAN SIGN CO INC,CASN,CAZ02A001,3277064,to install replacement signage for existing fr...,16.0,37017300.0,"2802 NOLENSVILLE PIKE\nNASHVILLE, TN 37211\n(3..."
3,2016029907,Building Residential - Roofing / Siding,Duplex,075150A13900CO,06/20/2016,06/20/2016,17648.0,139 STONERS GLEN DR,HERMITAGE,TN,37076,UNIT 139 STONERS GLEN PHASE 1,"MR ROOF NASHVILLE, LLC",CARK,CAA02R302,3251947,Remove existing shingles and replace with new ...,12.0,37015623.0,"139 STONERS GLEN DR\nHERMITAGE, TN 37076\n(36...."
4,201533533,Building Residential - Tenant Finish Out,"Multifamily, Tri-Plex, Quad, Apartments",10404024100,08/07/2015,11/17/2015,542696.0,70 MUSIC SQ W,NASHVILLE,TN,37203,LOT 1 O.B. HAYES FIRST PLAN CONSOLIDATION OF L...,Construction Enterprises Inc,CART,CAA03R398,2061885,SEE MASTER PERMIT T2015-26520...T.....GRAND AV...,19.0,37016400.0,"70 MUSIC SQ W\nNASHVILLE, TN 37203\n(36.147074..."
5,201605753,Building Residential - Rehab,Single Family Residence,131060E32200CO,02/10/2016,02/10/2016,50000.0,322 BOXMERE PL,NASHVILLE,TN,37215,UNIT 322 BURTON HILLS VILLAGE OF BOXMERE-PHASE 1,Sharp & Robbins Construction Llc,CARR,CAA01R301,2100747,to conduct interior renovation to existing uni...,25.0,37017902.0,"322 BOXMERE PL\nNASHVILLE, TN 37215\n(36.09425..."
6,2017032777,Building Residential - New,Single Family Residence,072103B00300CO,05/17/2017,05/24/2017,226201.0,1037 FAIRWIN AVE,NASHVILLE,TN,37216,UNIT 3 2619 GALLATIN PIKE TOWNHOMES,PARAGON GROUP LLC,CARN,CAA01R301,3367050,to construct 2008Sf single family residence wi...,5.0,37011400.0,"1037 FAIRWIN AVE\nNASHVILLE, TN 37216\n(36.198..."
7,201546095,Building Residential - Rehab,Single Family Residence,12809002600,10/22/2015,10/22/2015,30000.0,305 HARPETH VALLEY RD,NASHVILLE,TN,37221,LOT 35 B. V. 1776 ESTATES,"Odyssey Construction, Llc",CARR,CAA01R301,2078541,to make general repairs to existing residence....,22.0,37018301.0,"305 HARPETH VALLEY RD\nNASHVILLE, TN 37221\n(3..."
8,201619676,Building Commercial - Roofing / Siding,"Manufacturing, Light Hazardous (H1-H5)",09314033300,04/28/2016,04/28/2016,66961.0,625 FOGG ST,NASHVILLE,TN,37203,LOTS 30 TH 34 EWING & WETMORE,Ragan Construction Co Inc (Formerly Southern R...,CACK,CAH06H000,2119017,to reroof existing cromatics building.,19.0,37019500.0,"625 FOGG ST\nNASHVILLE, TN 37203\n(36.150909, ..."
9,201550728,Building Commercial - Foundation,"Multifamily, Apt / Twnhome > 5 Unit Bldg",08305009600,11/18/2015,02/01/2016,219566.0,1035 W EASTLAND AVE,NASHVILLE,TN,37206,"LOT 4 EDGEFIELD HOSPITAL, INC. RESUB",Southland Constructors Inc.,CACF,CAA03R399,2084584,"per shell permit 201550039, to conduct foundat...",5.0,37011700.0,"1035 W EASTLAND AVE\nNASHVILLE, TN 37206\n(36...."


### 
Building Residential - New: Permit Type = CARN

Building Commercial - New: Permit Type = CACN

Residential Rehab: Permit Type = CARR

Commercial Rehab: Permit Type = CACR

Building Demolition: Permit Type = CADM

In [7]:
building = bp.loc[bp['Permit Type Description'].str.contains('Building')]

In [8]:
new_building = building.loc[building['Permit Type Description'].str.contains('New')]

In [9]:
# Codes that we care about
new_residential = ['CAA01R301', 'CAA02R302', 'CAA03R298', 'CAA03R299', 'CAA03R301', 'CAA03R398', 'CAA03R399', 'CAA04R301', 'CAA05R202', 'CAA08R301', 'CAA10R301', 'CAZ08A001', 'DPLX2CNDO']

In [10]:
# Codes that we care about
new_commercial = ['CAA03R299', 'CAA03R301', 'CAA03R398', 'CAA03R399', 'CAA05R202']

In [11]:
rehab_commercial = ['CAA02R302', 'CAA03R201', 'CAA03R298', 'CAA03R299', 'CAA03R301', 'CAA03R398', 'CAA03R399']

In [12]:
demolition_codes = ['CAZ01A001', 'CAZ01A002']

In [13]:
commercial_rehab = bp.loc[(bp['Permit Type'] == 'CARR') & (bp['Permit Subtype'].isin(rehab_commercial))]

In [14]:
commercial_rehab.shape

(178, 20)

# Merge data based on parcel

In [15]:
matched = pd.merge(bp.loc[bp['Permit Subtype'].isin(['CAZ01A001', 'CAZ01A002'])], new_building.loc[new_building['Permit Subtype'].isin(new_commercial + new_residential)], on = 'Address')

In [16]:
matched.shape

(1060, 39)

In [17]:
# Residential Permits where one house was demolished and a new one was put up
res_rebuild = pd.merge(bp.loc[bp['Permit Subtype'] == 'CAZ01A001'], bp.loc[(bp['Permit Type'] == 'CARN') & (bp['Permit Subtype'].isin(new_residential))], on = 'Parcel')

In [18]:
res_rebuild.head()

,Permit #_x,Permit Type Description_x,Permit Subtype Description_x,Parcel,Date Entered_x,Date Issued_x,Const. Cost_x,Address_x,City_x,State_x,...,Zip_y,Subdivision/Lot_y,Contact_y,Permit Type_y,Permit Subtype_y,IVR Trk#_y,Purpose_y,Council Dist_y,Census Tract_y,Mapped Location_y
0,2016025907,Building Demolition Permit,Demolition Permit - Residential,10505038400,06/01/2016,06/02/2016,8000.0,1014 ARGYLE AVE,NASHVILLE,TN,...,37203,LOT 8 BRANSFORD RLTY CO SUB JONES & DUDLEY,LANDMARK HOMES OF TN INC,CARN,CAA01R301,3254241,to construct a single family residence with 28...,17.0,37016200.0,"1014 ARGYLE AVE\nNASHVILLE, TN 37203\n(36.1375..."
1,2016031224,Building Demolition Permit,Demolition Permit - Residential,08203030300,06/24/2016,06/24/2016,4000.0,1000 PENNOCK AVE,NASHVILLE,TN,...,37207,PT LOT 10 F E MANN PLAN,SELF CONTRACTOR RESIDENTIAL (SEE APPLICANT INF...,CARN,CAA01R301,3305330,REPLACE FIRE DAMAGED HOUSE...\nTO CONSTRUCT A ...,5.0,37011800.0,"1000 PENNOCK AVE\nNASHVILLE, TN 37207\n(36.188..."
2,2016031224,Building Demolition Permit,Demolition Permit - Residential,08203030300,06/24/2016,06/24/2016,4000.0,1000 PENNOCK AVE,NASHVILLE,TN,...,37207,PT LOT 10 F E MANN PLAN,C & O Construction Co. Llc,CARN,CAA01R301,2104734,to rehab and convert existing 600 sf second fl...,5.0,37011800.0,"1000B PENNOCK AVE\nNASHVILLE, TN 37207\n(36.18..."
3,2016049441,Building Demolition Permit,Demolition Permit - Residential,09115028700,09/15/2016,09/16/2016,7500.0,4906 NEVADA AVE,NASHVILLE,TN,...,37209,PT BLK 35 CHARLOTTE PARK CO FIRST ADDN,ASPEN CONSTRUCTION INC,CARN,CAA01R301,3277281,Need demo permit 2016049441 completed prior to...,24.0,37013400.0,"4906 NEVADA AVE\nNASHVILLE, TN 37209\n(36.1485..."
4,2017027889,Building Demolition Permit,Demolition Permit - Residential,13112009500,04/26/2017,04/26/2017,12000.0,1031 MILESDALE DR,NASHVILLE,TN,...,37204,LOT 15 LEALAND HGTS,"SAVAGE CONSTRUCTION GROUP, LLC",CARN,CAA01R301,3350935,TO CONSTRUCT A NEW SINGLE FAMILY RES AT 3581 S...,25.0,37017702.0,"1031 MILESDALE DR\nNASHVILLE, TN 37204\n(36.09..."


In [19]:
os.getcwd()

'C:\\Users\\chris\\data_science_projects\\data-question-4-affordable-housing-green-dogs'

In [20]:
sf_2009 = pd.read_csv('2009_SINGLE_FAMILY.txt')

In [21]:
sf_2013 = pd.read_csv('2013_SINGLE_FAMILY.txt')

In [22]:
sf_2017 = pd.read_csv('2017_SINGLE_FAMILY.txt')

In [23]:
sf1 = pd.merge(sf_2009, sf_2013, on= 'AddressFullAddress', how = 'outer',suffixes = ('_f','_s'))
sf1.columns

sf1.rename(columns = {'FinishedArea_f' : '2009_FinishedArea', 'FinishedArea_s' : '2013_FinishedArea'}, inplace = True)

sf1['APN']= sf1['APN_f'].where(sf1['APN_s'].isnull(), sf1['APN_s'])
sf1['DistrictCode']= sf1['DistrictCode_f'].where(sf1['DistrictCode_s'].isnull(), sf1['DistrictCode_s'])
sf1['Council District']= sf1['Council District_f'].where(sf1['Council District_s'].isnull(), sf1['Council District_s'])
sf1['AddressCity']= sf1['AddressCity_f'].where(sf1['AddressCity_s'].isnull(), sf1['AddressCity_s'])
sf1['AddressPostalCode']= sf1['AddressPostalCode_f'].where(sf1['AddressPostalCode_s'].isnull(), sf1['AddressPostalCode_s'])
#sf1['FinishedArea']= sf1['FinishedArea_f'].where(sf1['FinishedArea_s'].isnull(), sf1['FinishedArea_s'])
sf1.columns

sf1.drop(['APN_f','DistrictCode_f','Council District_f','AddressCity_f',
          'AddressPostalCode_f','DistrictCode_s','Council District_s','AddressCity_s','APN_s','AddressPostalCode_s'], axis = 1,inplace = True)

final_sf = pd.merge(sf1, sf_2017,on= 'AddressFullAddress', how = 'outer', suffixes = ('_m','_n'),indicator = True)
final_sf.columns

final_sf.rename(columns = {'FinishedArea': '2017_FinishedArea'}, inplace = True)

final_sf['APN']= final_sf['APN_m'].where(final_sf['APN_n'].isnull(), final_sf['APN_n'])
final_sf['DistrictCode']= final_sf['DistrictCode_m'].where(final_sf['DistrictCode_n'].isnull(), final_sf['DistrictCode_n'])
final_sf['Council District']= final_sf['Council District_m'].where(final_sf['Council District_n'].isnull(), final_sf['Council District_n'])
final_sf['AddressCity']= final_sf['AddressCity_m'].where(final_sf['AddressCity_n'].isnull(), final_sf['AddressCity_n'])
final_sf['AddressPostalCode']= final_sf['AddressPostalCode_m'].where(final_sf['AddressPostalCode_n'].isnull(), final_sf['AddressPostalCode_n'])
#final_sf['FinishedArea']= final_sf['FinishedArea_m'].where(final_sf['FinishedArea_n'].isnull(), final_sf['FinishedArea_n'])


final_sf.drop(['APN_m','DistrictCode_m','Council District_m','AddressCity_m',
          'AddressPostalCode_m','DistrictCode_n','Council District_n','AddressCity_n','APN_n','AddressPostalCode_n'], axis = 1, inplace = True)
final_sf.columns

final_sf.drop_duplicates(keep ='first').shape

(201854, 19)

In [24]:
sf1

,AddressFullAddress,2009 LAND,2009 IMPR,2009 TOTAL APPR,2009 TOTAL ASSD,2013 LAND,2013 IMPR,2013 TOTAL APPR,2013 TOTAL ASSD,APN,DistrictCode,Council District,AddressCity,AddressPostalCode
0,1012 OLD BUCK HILL RD,49800.0,65700.0,115500.0,28875.0,43500.0,70500.0,114000.0,28500.0,00200000300,RT,10.0,GOODLETTSVILLE,37072.0
1,1109 HIGHLAND AVE,44100.0,88700.0,132800.0,33200.0,40100.0,96500.0,136600.0,34150.0,00200000700,RT,10.0,GOODLETTSVILLE,37072.0
2,1992 SPRINGFIELD HWY,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,00200001001,GSD,10.0,GOODLETTSVILLE,37072.0
3,0 SPRINGFIELD HWY,15000.0,137900.0,152900.0,38225.0,15000.0,158400.0,173400.0,43350.0,00200001001002,GSD,10.0,GOODLETTSVILLE,37072.0
4,1855 LAKE RD,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,00200001200,GSD,10.0,RIDGETOP,37152.0
5,133 EAST END RD,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,00200001400,GSD,10.0,GOODLETTSVILLE,37072.0
6,133 B EAST END RD,41100.0,27000.0,68100.0,17025.0,32300.0,26900.0,59200.0,14800.0,00200001400001,RT,10.0,GOODLETTSVILLE,37072.0
7,141 EAST END RD,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,00200001600,GSD,10.0,GOODLETTSVILLE,37072.0
8,141 B EAST END RD,25100.0,60900.0,86000.0,21500.0,19700.0,63300.0,83000.0,20750.0,00200001600001,RT,10.0,GOODLETTSVILLE,37072.0
9,226 EAST END RD,41800.0,129800.0,171600.0,42900.0,41800.0,140600.0,182400.0,45600.0,00200002400,GSD,10.0,GOODLETTSVILLE,37072.0


In [25]:
final_sf.shape

(219618, 19)

In [26]:
final_sf.columns

Index(['AddressFullAddress', '2009 LAND', '2009 IMPR', '2009 TOTAL APPR',
       '2009 TOTAL ASSD', '2013 LAND', '2013 IMPR', '2013 TOTAL APPR',
       '2013 TOTAL ASSD', '2017 LAND', '2017 IMPR', '2017 TOTAL APPR',
       '2017 TOTAL ASSD', '_merge', 'APN', 'DistrictCode', 'Council District',
       'AddressCity', 'AddressPostalCode'],
      dtype='object')

In [29]:
r = 0.04 #Annual interest rate
y = 30 #Length of mortgage (years)
n = 12*y # Number of monthly payments
d = 0.05 # Down payment percentage
final_sf['Mortgage_Per_Year_2009'] = 12* (1-d)*final_sf['2009 TOTAL APPR'] * r/12 *(1+r/12)**n / ((1+r/12)**n -1)
final_sf['Mortgage_Per_Year_2013'] = 12* (1-d)*final_sf['2013 TOTAL APPR'] * r/12 *(1+r/12)**n / ((1+r/12)**n -1)
final_sf['Mortgage_Per_Year_2017'] = 12* (1-d)*final_sf['2017 TOTAL APPR'] * r/12 *(1+r/12)**n / ((1+r/12)**n -1)

In [30]:
final_sf

,AddressFullAddress,2009 LAND,2009 IMPR,2009 TOTAL APPR,2009 TOTAL ASSD,2013 LAND,2013 IMPR,2013 TOTAL APPR,2013 TOTAL ASSD,2017 LAND,...,2017 TOTAL ASSD,_merge,APN,DistrictCode,Council District,AddressCity,AddressPostalCode,Mortgage_Per_Year_2009,Mortgage_Per_Year_2013,Mortgage_Per_Year_2017
0,1012 OLD BUCK HILL RD,49800.0,65700.0,115500.0,28875.0,43500.0,70500.0,114000.0,28500.0,46700.0,...,33850.0,both,00200000300,RT,10.0,GOODLETTSVILLE,37072.0,6286.127195,6204.489180,7369.191535
1,1109 HIGHLAND AVE,44100.0,88700.0,132800.0,33200.0,40100.0,96500.0,136600.0,34150.0,40100.0,...,37450.0,both,00200000700,RT,10.0,GOODLETTSVILLE,37072.0,7227.685641,7434.501947,8152.916484
2,1992 SPRINGFIELD HWY,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,left_only,00200001001,GSD,10.0,GOODLETTSVILLE,37072.0,0.000000,NaN,NaN
3,0 SPRINGFIELD HWY,15000.0,137900.0,152900.0,38225.0,15000.0,158400.0,173400.0,43350.0,18000.0,...,50775.0,both,00200001001002,GSD,10.0,GOODLETTSVILLE,37072.0,8321.635049,9437.354595,11053.787302
4,1855 LAKE RD,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,left_only,00200001200,GSD,10.0,RIDGETOP,37152.0,0.000000,NaN,NaN
5,133 EAST END RD,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,left_only,00200001400,GSD,10.0,GOODLETTSVILLE,37072.0,0.000000,NaN,NaN
6,133 B EAST END RD,41100.0,27000.0,68100.0,17025.0,32300.0,26900.0,59200.0,14800.0,30000.0,...,16025.0,both,00200001400001,RT,10.0,GOODLETTSVILLE,37072.0,3706.365905,3221.980346,3488.664530
7,141 EAST END RD,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,left_only,00200001600,GSD,10.0,GOODLETTSVILLE,37072.0,0.000000,NaN,NaN
8,141 B EAST END RD,25100.0,60900.0,86000.0,21500.0,19700.0,63300.0,83000.0,20750.0,18300.0,...,24450.0,both,00200001600001,RT,10.0,GOODLETTSVILLE,37072.0,4680.579557,4517.303526,5322.798612
9,226 EAST END RD,41800.0,129800.0,171600.0,42900.0,41800.0,140600.0,182400.0,45600.0,45200.0,...,52900.0,both,00200002400,GSD,10.0,GOODLETTSVILLE,37072.0,9339.388976,9927.182688,11516.402723


In [33]:
# Total number of single family homes in 2017
final_sf['2017 TOTAL APPR'].count()

215349

In [34]:
#Percent of Affordable Homes in 2017
final_sf.loc[final_sf.Mortgage_Per_Year_2017 < 12240].AddressFullAddress.count()/final_sf['2017 TOTAL APPR'].count()

0.5931070030508616

In [35]:
#Percent of affordable homes in 2009
final_sf.loc[final_sf.Mortgage_Per_Year_2009 < 11550].AddressFullAddress.count()/final_sf['2009 TOTAL APPR'].count()

0.7579259009466945

In [36]:
final_sf.loc[final_sf.Mortgage_Per_Year_2017 < 12240].AddressFullAddress.count()


127725

In [37]:
# Change in the number of affordable homes from 2009 to 2017
final_sf.loc[final_sf.Mortgage_Per_Year_2009 < 11550].AddressFullAddress.count() - final_sf.loc[final_sf.Mortgage_Per_Year_2017 < 12240].AddressFullAddress.count()

25110

In [27]:
hi_2017 = pd.read_csv('ACS_17_1YR_S1901_with_ann.csv')

In [28]:
hi_2017.head()

,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC01,HC01_MOE_VC01,HC02_EST_VC01,HC02_MOE_VC01,HC03_EST_VC01,HC03_MOE_VC01,HC04_EST_VC01,...,HC04_EST_VC19,HC04_MOE_VC19,HC01_EST_VC20,HC01_MOE_VC20,HC02_EST_VC20,HC02_MOE_VC20,HC03_EST_VC20,HC03_MOE_VC20,HC04_EST_VC20,HC04_MOE_VC20
0,Id,Id2,Geography,Households; Estimate; Total,Households; Margin of Error; Total,Families; Estimate; Total,Families; Margin of Error; Total,Married-couple families; Estimate; Total,Married-couple families; Margin of Error; Total,Nonfamily households; Estimate; Total,...,Nonfamily households; Estimate; PERCENT ALLOCA...,Nonfamily households; Margin of Error; PERCENT...,Households; Estimate; PERCENT ALLOCATED - Nonf...,Households; Margin of Error; PERCENT ALLOCATED...,Families; Estimate; PERCENT ALLOCATED - Nonfam...,Families; Margin of Error; PERCENT ALLOCATED -...,Married-couple families; Estimate; PERCENT ALL...,Married-couple families; Margin of Error; PERC...,Nonfamily households; Estimate; PERCENT ALLOCA...,Nonfamily households; Margin of Error; PERCENT...
1,0500000US47037,47037,"Davidson County, Tennessee",283929,3870,161513,4639,112277,4229,122416,...,(X),(X),(X),(X),(X),(X),(X),(X),35.7,(X)


In [45]:
from geopy.geocoders import Nominatim
import numpy as np

In [41]:
final_sf['Address'] = final_sf['AddressFullAddress'] + ' Davidson County , TN'

In [43]:
def get_lat(x):
   try:
       value = geolocator.geocode(x).latitude
   except:
       value = np.nan
   return value

In [46]:
final_sf.iloc[0:10,'lat'] = final_sf.iloc[0:10].Address.apply(get_lat)

ValueError: Can only index by location with a [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array]

In [51]:
final_sf['Pct_Change 2013-2017'] = (final_sf['2017 TOTAL APPR'] - final_sf['2013 TOTAL APPR']) / final_sf['2013 TOTAL APPR']
final_sf['Pct_Change_Overall'] = (final_sf['2017 TOTAL APPR'] - final_sf['2009 TOTAL APPR']) / final_sf['2009 TOTAL APPR'] 

In [55]:
final_sf.groupby('Council District').Pct_Change_Overall.mean().nlargest(6) 

Council District
15.0         inf
5.0     1.352403
6.0     1.045688
7.0     0.904015
17.0    0.903942
20.0    0.825503
Name: Pct_Change_Overall, dtype: float64

In [53]:
final_sf.loc[final_sf['Council District'] == 15]

,AddressFullAddress,2009 LAND,2009 IMPR,2009 TOTAL APPR,2009 TOTAL ASSD,2013 LAND,2013 IMPR,2013 TOTAL APPR,2013 TOTAL ASSD,2017 LAND,...,DistrictCode,Council District,AddressCity,AddressPostalCode,Mortgage_Per_Year_2009,Mortgage_Per_Year_2013,Mortgage_Per_Year_2017,Address,Pct_Change 2013-2017,Pct_Change_Overall
18173,2505 PENNINGTON BEND RD,37500.0,68900.0,106400.0,26600.0,30000.0,78500.0,108500.0,27125.0,40000.0,...,USD,15.0,NASHVILLE,37214.0,5790.856568,5905.149790,8512.123752,"2505 PENNINGTON BEND RD Davidson County , TN",0.441475,0.469925
18174,2507 PENNINGTON BEND RD,37500.0,88300.0,125800.0,31450.0,30000.0,87400.0,117400.0,29350.0,60000.0,...,USD,15.0,NASHVILLE,37214.0,6846.708235,6389.535348,10182.981803,"2507 PENNINGTON BEND RD Davidson County , TN",0.593697,0.487281
18175,2511 PENNINGTON BEND RD,100100.0,68900.0,169000.0,42250.0,55400.0,89100.0,144500.0,36125.0,110800.0,...,USD,15.0,NASHVILLE,37214.0,9197.883082,7864.462162,13018.542209,"2511 PENNINGTON BEND RD Davidson County , TN",0.655363,0.415385
18176,2511 PENNINGTON BEND RD,37500.0,86200.0,123700.0,30925.0,55400.0,89100.0,144500.0,36125.0,110800.0,...,USD,15.0,NASHVILLE,37214.0,6732.415014,7864.462162,13018.542209,"2511 PENNINGTON BEND RD Davidson County , TN",0.655363,0.933711
18177,2513 PENNINGTON BEND RD,37500.0,73300.0,110800.0,27700.0,30000.0,62700.0,92700.0,23175.0,40000.0,...,USD,15.0,NASHVILLE,37214.0,6030.328080,5045.229359,6460.288295,"2513 PENNINGTON BEND RD Davidson County , TN",0.280475,0.071300
18178,2515 PENNINGTON BEND RD,67200.0,137400.0,204600.0,51150.0,43200.0,106200.0,149400.0,37350.0,86400.0,...,USD,15.0,NASHVILLE,37214.0,11135.425318,8131.146346,10912.281408,"2515 PENNINGTON BEND RD Davidson County , TN",0.342035,-0.020039
18184,2517 PENNINGTON BEND RD,137600.0,142200.0,279800.0,69950.0,74500.0,151600.0,226100.0,56525.0,149000.0,...,USD,15.0,NASHVILLE,37214.0,15228.211163,12305.570207,18624.352608,"2517 PENNINGTON BEND RD Davidson County , TN",0.513490,0.223016
19805,2641 MIAMI AVE,70200.0,118300.0,188500.0,47125.0,NaN,NaN,NaN,NaN,NaN,...,USD,15.0,NASHVILLE,37214.0,10259.177284,NaN,NaN,"2641 MIAMI AVE Davidson County , TN",NaN,NaN
19806,2645 MIAMI AVE,70200.0,106900.0,177100.0,44275.0,25000.0,116100.0,141100.0,35275.0,30000.0,...,USD,15.0,NASHVILLE,37214.0,9638.728366,7679.415994,11091.885043,"2645 MIAMI AVE Davidson County , TN",0.444366,0.150762
19807,2651 MIAMI AVE,70200.0,115300.0,185500.0,46375.0,25000.0,99500.0,124500.0,31125.0,30000.0,...,USD,15.0,NASHVILLE,37214.0,10095.901253,6775.955289,8528.451355,"2651 MIAMI AVE Davidson County , TN",0.258635,-0.155256
